We import the necessary modules

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from numba import jit


# We had a problem with the path of the other module
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))

Definition of the constants and the initial fields

In [ ]:
#############
# Constants #
#############

nb_points = 42

R = 8.314  # Ideal gas constant in J/(mol* K)

M_N2 = 0.02802  # kg/mol

M_air = 0.02895  # kg/mol

M_CH4 = 0.016042  # kg/mol

dt = 1e-6

final_time = 1

nb_timesteps = int(final_time / dt)

Lx = 2e-3

Ly = 2e-3

L_slot = 0.5e-3

L_coflow = 0.5e-3

U_slot = 1.0

T_slot = 300

U_coflow = 0.2

T_coflow = 300

rho = 1.1614 # Fluid density

dx = Lx / (nb_points - 1)

dy = Ly / (nb_points - 1)

nu = 15e-6

tolerance_sor = 1e-7 # Tolerance for the convergence of the SOR algorithm

tolerance_sys = 1e-4 # Tolerance for the convergence of the whole system

max_iter = 10000  # Maximum number of iterations for achieving convergence

omega = 1.5  # Parameter for the Successive Overrelaxation Method (SOR), it has to be between 1 and 2


##################
# Initial fields #
##################

# Velocity fields
v = np.zeros((nb_points, nb_points))
u = np.zeros((nb_points, nb_points))

# Pressure field
P = np.ones((nb_points, nb_points))

# Species field (nitrogen)
Y_n2 = np.zeros((nb_points, nb_points))

In [ ]:
#################################################
# Derivatives and second derivatives definition #
#################################################

# Attention: the sense of the derivatives is inversed!!

#@jit(fastmath=True, nopython=True)
def derivative_x_upwind(vel: np.array, a:np.array, dx=dx) -> np.array:
    dvel_dx = np.zeros_like(vel)

    # For interior points, choose upwind direction based on the sign of u
    for i in range(1, vel.shape[1] - 2): # Skip the boundaries
        dvel_dx[:, i] = np.where(a[:, i] >= 0, (vel[:, i] - vel[:, i - 1]) / dx, (vel[:, i + 1] - vel[:, i]) / dx)

    return dvel_dx


#@jit(fastmath=True, nopython=True)
def derivative_y_upwind(vel: np.array, a:np.array, dy=dy) -> np.array:
    dvel_dy = np.zeros_like(vel)

    # For interior points, choose upwind direction based on the sign of vel
    for i in range(1, vel.shape[0] - 2): 
        # Differenciate between positive and negative flow
        dvel_dy[i, :] = np.where(a[i, :] >= 0, (vel[i, :] - vel[i - 1, :]) / dy, (vel[i + 1, :] - vel[i, :]) / dy)

    return dvel_dy


#@jit(fastmath=True, nopython=True)
def derivative_x_centered(u: np.array, dx=dx) -> np.array:
    du_dx = np.zeros_like(u)

    for i in range(2, nb_points - 2):
        for j in range(2, nb_points - 2):
            du_dx[i, j] = (u[i, j+1] - u[i, j-1]) / (2 * dx)  # Central difference

    return du_dx


#@jit(fastmath=True, nopython=True)
def derivative_y_centered(u: np.array, dy=dy) -> np.array:
    du_dy = np.zeros_like(u)

    for i in range(2, nb_points - 2):
        for j in range(2, nb_points - 2):
            du_dy[i, j] = (u[i+1, j] - u[i-1, j]) / (2 * dy)  # Central difference for interior points

    return du_dy


#@jit(fastmath=True, nopython=True)
def second_centered_x(u: np.array, dx=dx) -> np.array:

    for i in range(2, nb_points - 2):
        for j in range(2, nb_points - 2):
            u_left = u[i, j-1]
            u_right = u[i, j+1]
            u[i, j] = (u_right - 2 * u [i, j] + u_left) / dx**2

    return u


#@jit(fastmath=True, nopython=True)
def second_centered_y(u: np.array, dy=dy) -> np.array:

    for i in range(2, nb_points - 2):
        for j in range(2, nb_points - 2):
            u_left = u[i-1, j]
            u_right = u[i+1, j]
            u[i, j] = (u_right - 2 * u [i, j] + u_left) / dy**2

    return u

Next, we define the Successive Overrelaxation method

In [ ]:
#@jit(fastmath=True, nopython=True)
def sor(P, f, tolerance_sor=tolerance_sor, max_iter=max_iter, omega=omega):
    """
    Successive Overrelaxation (SOR) method for solving the pressure Poisson equation.
    Optimized using Numba

    Parameters:
        P (np.array): Initial guess for the pressure field.
        f (np.array): Right-hand side of the Poisson equation.
        tolerance (float): Convergence tolerance for the iterative method.
        max_iter (int): Maximum number of iterations.
        omega (float): Relaxation factor, between 1 and 2.

    Returns:
        np.array: Updated pressure field.
    """

    coef = 2 * (1 / dx**2 + 1 / dy**2)

    for _ in range(max_iter):
        P_old = P.copy()
        laplacian_P = P.copy()
        
        for i in range(2, nb_points - 2):
            for j in range(2, nb_points - 2):
                laplacian_P[i, j] = (P_old[i+1, j] + P[i-1, j]) / dx**2 + (P_old[i, j+1] + P[i, j-1]) / dy**2
                
                # Update P using the SOR formula
                P[i, j] = (1 - omega) * P_old[i, j] + (omega / coef) * (laplacian_P[i, j] - f[i, j])
        
        # Compute the residual to check for convergence
        residual = np.linalg.norm(P - P_old, ord=2)
        if np.linalg.norm(P_old, ord=2) > 10e-10:  # Avoid divide-by-zero by checking the norm
            residual /= np.linalg.norm(P_old, ord=2)

        if residual < tolerance_sor:
            break

    return P